In [2]:
from pathlib import Path
import sys
import geopandas as gpd

# Project root - use explicit path if auto-detect fails (e.g. kernel cwd invalid)
base = Path("/Users/amandaeames/Documents/gitrepo/data-research")
if not (base / "reviews/cl-ocha-ab").exists():
    try:
        base = Path.cwd()
        if not (base / "reviews/cl-ocha-ab").exists():
            base = Path.cwd().resolve().parents[4]
    except FileNotFoundError:
        pass
if not (base / "reviews/cl-ocha-ab").exists():
    raise FileNotFoundError(f"Data not found under {base}. Set base = Path('/path/to/data-research')")
cities = gpd.read_file(base / "reviews/cl-ocha-ab/releases/2021/sample/cl_admin_locode.gpkg")
ecoregions = gpd.read_file(base / "reviews/resolve-ecoregions/releases/2017/sample/ecoregions_chile.gpkg")
cities = cities.to_crs(ecoregions.crs)

In [3]:
# Spatial join: each city gets all ecoregions it touches
joined = gpd.sjoin(cities, ecoregions[["biome_name", "eco_name", "geometry"]], how="left", predicate="intersects")

In [4]:
# All biomes per city (some cities span multiple biomes)
biomes_per_city = joined.groupby("comuna_name", dropna=False)["biome_name"].apply(lambda x: sorted(x.dropna().unique().tolist())).reset_index()
biomes_per_city.columns = ["comuna_name", "all_biomes"]
biomes_per_city

,comuna_name,all_biomes
0,Algarrobo,"Mediterranean Forests, Woodlands & Scrub"
1,Alhué,"Mediterranean Forests, Woodlands & Scrub"
2,Alto Biobío,Temperate Broadleaf & Mixed Forests
3,Alto Hospicio,Deserts & Xeric Shrublands
4,Alto del Carmen,"Mediterranean Forests, Woodlands & Scrub, Mont..."
...,...,...
339,Yumbel,"Mediterranean Forests, Woodlands & Scrub, Temp..."
340,Yungay,"Mediterranean Forests, Woodlands & Scrub, Temp..."
341,Zapallar,"Mediterranean Forests, Woodlands & Scrub"
342,Ñiquén,"Mediterranean Forests, Woodlands & Scrub, Temp..."


In [5]:
# Main biome = the one covering the largest area of each city
overlap = gpd.overlay(
    cities[["comuna_name", "locode", "geometry"]],
    ecoregions[["biome_name", "geometry"]],
    how="intersection"
)
overlap["area_km2"] = overlap.to_crs(epsg=32719).area / 1e6
main = overlap.loc[overlap.groupby("comuna_name")["area_km2"].idxmax()][["comuna_name", "locode", "biome_name"]]
main

,comuna_name,locode,biome_name
97,Algarrobo,CL ALG,"Mediterranean Forests, Woodlands & Scrub"
464,Alhué,CL ALH,"Mediterranean Forests, Woodlands & Scrub"
269,Alto Biobío,NaN,Temperate Broadleaf & Mixed Forests
1,Alto Hospicio,CL AHP,Deserts & Xeric Shrublands
42,Alto del Carmen,CL ADC,Montane Grasslands & Shrublands
...,...,...,...
267,Yumbel,CL YUM,"Mediterranean Forests, Woodlands & Scrub"
511,Yungay,CL YUN,"Mediterranean Forests, Woodlands & Scrub"
88,Zapallar,NaN,"Mediterranean Forests, Woodlands & Scrub"
528,Ñiquén,CL NQN,"Mediterranean Forests, Woodlands & Scrub"


In [6]:
# Merge all biomes + main biome for each city
result = main.merge(biomes_per_city, on="comuna_name", how="left")
result = result.rename(columns={"biome_name": "main_biome"})
result

,comuna_name,locode,main_biome,all_biomes
0,Algarrobo,CL ALG,"Mediterranean Forests, Woodlands & Scrub","Mediterranean Forests, Woodlands & Scrub"
1,Alhué,CL ALH,"Mediterranean Forests, Woodlands & Scrub","Mediterranean Forests, Woodlands & Scrub"
2,Alto Biobío,NaN,Temperate Broadleaf & Mixed Forests,Temperate Broadleaf & Mixed Forests
3,Alto Hospicio,CL AHP,Deserts & Xeric Shrublands,Deserts & Xeric Shrublands
4,Alto del Carmen,CL ADC,Montane Grasslands & Shrublands,"Mediterranean Forests, Woodlands & Scrub, Mont..."
...,...,...,...,...
337,Yumbel,CL YUM,"Mediterranean Forests, Woodlands & Scrub","Mediterranean Forests, Woodlands & Scrub, Temp..."
338,Yungay,CL YUN,"Mediterranean Forests, Woodlands & Scrub","Mediterranean Forests, Woodlands & Scrub, Temp..."
339,Zapallar,NaN,"Mediterranean Forests, Woodlands & Scrub","Mediterranean Forests, Woodlands & Scrub"
340,Ñiquén,CL NQN,"Mediterranean Forests, Woodlands & Scrub","Mediterranean Forests, Woodlands & Scrub, Temp..."


In [7]:
result.to_csv("../sample/cl_main_biome.csv", index=False)